In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection = dict(database='default',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student',
                  password='dpo_python_2020')

In [3]:
query = '''
SELECT 
    cohort, 
    day_distance, 
    uniqExact(DeviceID) as devices
FROM
    (
    SELECT 
        l.AppPlatform as AppPlatform, 
        l.events as events, 
        CAST(l.EventDate as Date) as EventDate, 
        l.DeviceID as DeviceID, 
        r.birthday as birthday, 
        CAST(l.EventDate as Date)  - r.birthday as day_distance
    FROM events as l
    LEFT JOIN
        (
        SELECT 
            DeviceID, 
            MIN(CAST(EventDate as Date)) as birthday
        FROM events 
        WHERE AppPlatform == 'iOS'
        GROUP BY DeviceID
        ) as r
            ON l.DeviceID = r.DeviceID
    WHERE AppPlatform == 'iOS'
    )
WHERE day_distance <= 30
GROUP BY 
    birthday as cohort, 
    day_distance
ORDER BY
    cohort ASC, 
    day_distance ASC 
'''

In [5]:
retention = ph.read_clickhouse(query=query, connection=connection)

In [8]:
retention.head()

,cohort,day_distance,devices
0,2019-01-01,0,191013
1,2019-01-01,1,71636
2,2019-01-01,2,69761
3,2019-01-01,3,65577
4,2019-01-01,4,63249
